In [106]:
import os
os.environ['DISPLAY'] = ':1'
import yaml
import open3d as o3d
import copy
import numpy as np
import functools


In [41]:
color_candidates = [
    [255, 0, 0],  # Red
    [0, 255, 0],  # Green
    [0, 0, 255],  # Blue
    [255, 255, 0],  # Yellow
    [0, 255, 255],  # Cyan
    [255, 0, 255],  # Magenta
    [255, 255, 255],  # White
    [128, 128, 128],  # Gray
    [255, 165, 0],  # Orange
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 0],  # Olive
]

path = 'data_rlbench/urdf/basketball_in_hoop/basket_ball_hoop_visual.ply'
mesh = o3d.io.read_triangle_mesh(path)
o3d.visualization.draw_geometries([mesh])

In [ ]:
cfg_idx = 3
meshes = []
all_triangle_indices = []

while True:
    cfg_path = f'data_rlbench/urdf/basketball_in_hoop/basket_ball_hoop_visual_{cfg_idx}.yaml'
    if not os.path.exists(cfg_path):
        break
    
    cfg = yaml.load(open(cfg_path), Loader=yaml.FullLoader)
    indices = np.array(cfg['indices']).reshape(-1, 3)
    colors = np.array(cfg['colors']).reshape(-1, 3)
    print(colors[0])

    recorded_triangle_indices = np.where((np.asarray(mesh.triangles)[:, None, :] == indices[None,:, :]).all(-1).any(-1))[0]
    mesh1 = copy.deepcopy(mesh)
    mesh1.triangles = o3d.utility.Vector3iVector(np.asarray(mesh1.triangles)[recorded_triangle_indices])
    # mesh1.compute_vertex_normals()
    mesh1.paint_uniform_color(color_candidates[cfg_idx % len(color_candidates)])
    meshes.append(mesh1)
    all_triangle_indices.append(recorded_triangle_indices)
    break

    cfg_idx += 1


all_triangle_indices = np.unique(np.concatenate(all_triangle_indices))
uncovered_indices = np.setdiff1d(np.arange(len(mesh.triangles)), all_triangle_indices)

if len(uncovered_indices) > 0 and False:
    mesh1 = copy.deepcopy(mesh)
    mesh1.triangles = o3d.utility.Vector3iVector(np.asarray(mesh1.triangles)[uncovered_indices])
    # mesh1.compute_vertex_normals()
    meshes.append(mesh1)


o3d.visualization.draw_geometries(meshes)

[0.75 0.25 0.25]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


In [107]:

cfg_idx = 0
meshes = []

while True:
    cfg_path = f'data_rlbench/urdf/basketball_in_hoop/basket_ball_hoop_visual_{cfg_idx}.yaml'
    if not os.path.exists(cfg_path):
        break
    
    cfg = yaml.load(open(cfg_path), Loader=yaml.FullLoader)
    indices = np.array(cfg['indices']).reshape(-1, 3)
    colors = np.array(cfg['colors']).reshape(-1, 3)
    vertices = np.array(cfg['vertices']).reshape(-1, 3)
    print(colors[0])

    recorded_triangle_indices = np.where((np.asarray(mesh.triangles)[:, None, :] == indices[None,:, :]).all(-1).any(-1))[0]
    triangle_vertice_indices = np.asarray(mesh.triangles)[recorded_triangle_indices].flatten()
    
    recorded_vertices_indices = np.where((np.isclose(np.asarray(mesh.vertices)[:, None, :], vertices[None,:, :])).all(-1).any(-1))[0]
    
    vertices_indices = np.unique(np.concatenate([triangle_vertice_indices, recorded_vertices_indices]))
    
    triangle_indices = np.where((np.asarray(mesh.triangles)[:, None, :] == vertices_indices[None,:, None]).any(1).all(-1))[0]
    
    mesh1 = copy.deepcopy(mesh)
    mesh1.triangles = o3d.utility.Vector3iVector(np.asarray(mesh1.triangles)[triangle_indices])
    # mesh1.compute_vertex_normals()
    mesh1.paint_uniform_color(colors[0])
    meshes.append(mesh1)
    
    cfg_idx += 1

o3d.visualization.draw_geometries(meshes)

[0.83999997 0.83999997 0.83999997]
[1. 1. 1.]
[0.75 0.25 0.25]
[0.75 0.25 0.25]
[1. 1. 1.]


In [108]:
combined_mesh = functools.reduce(lambda x, y: x + y, meshes)
o3d.io.write_triangle_mesh('combined_mesh.obj', combined_mesh)

True